In [1]:
import polars as pl
import psycopg2

from psycopg2.extras import RealDictCursor

pl.Config(tbl_rows=60)

In [2]:
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="test_crypto_pricing",
    user="postgres",
    password="admin",
)

In [3]:
cur = conn.cursor()

cur.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
    AND table_type = 'BASE TABLE';
""")

cur.fetchall()

[('market_trade',), ('orderbook_updates',), ('orderbook_levels',)]

In [24]:
cur = conn.cursor(cursor_factory=RealDictCursor)

cur.execute("select * from orderbook_updates")

pl.DataFrame(cur.fetchall()).filter(pl.col("symbol") == "ETHUSDT").head(5)

ob_update_id,event_time,transaction_time,symbol,first_update_id,last_update_id,previous_update_id
i64,"datetime[μs, UTC]","datetime[μs, UTC]",str,i64,i64,i64
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995
4,2025-12-05 21:49:21.051 UTC,2025-12-05 21:49:21.049 UTC,"""ETHUSDT""",9388388949361,9388388969701,9388388948924
11,2025-12-05 21:49:21.159 UTC,2025-12-05 21:49:21.157 UTC,"""ETHUSDT""",9388388970939,9388388980039,9388388969701
18,2025-12-05 21:49:21.267 UTC,2025-12-05 21:49:21.265 UTC,"""ETHUSDT""",9388388980264,9388388995726,9388388980039
23,2025-12-05 21:49:21.375 UTC,2025-12-05 21:49:21.373 UTC,"""ETHUSDT""",9388388996316,9388389011714,9388388995726


In [21]:
cur = conn.cursor(cursor_factory=RealDictCursor)

cur.execute("select * from orderbook_levels limit 100")

pl.DataFrame(cur.fetchall()).head(40)

ob_update_id,side,level_id,price,quantity
i64,i64,i64,"decimal[*,10]","decimal[*,10]"
1,1,1,2.0234000000,43230.2000000000
1,1,2,2.0233000000,29198.1000000000
1,1,3,2.0232000000,18808.8000000000
1,1,4,2.0231000000,37606.5000000000
1,1,5,2.0230000000,19238.5000000000
1,1,6,2.0229000000,8655.2000000000
1,1,7,2.0228000000,12153.6000000000
1,1,8,2.0227000000,32815.2000000000
1,1,9,2.0226000000,13035.5000000000


In [22]:
cur = conn.cursor(cursor_factory=RealDictCursor)
conn.rollback()

cur.execute(
    """
    select count(*) from orderbook_updates
    """
)

pl.DataFrame(cur.fetchall())

count
i64
5862


In [23]:
cur = conn.cursor(cursor_factory=RealDictCursor)
conn.rollback()

cur.execute(
    """
    select * from orderbook_updates as obu 
    left join orderbook_levels as obl on obu.ob_update_id = obl.ob_update_id
    where symbol = 'ETHUSDT'
    limit 1000
    """
)

pl.DataFrame(cur.fetchall()).sort(
    by=["ob_update_id", "level_id", "side"], descending=[False, False, True]
).head(50)

ob_update_id,event_time,transaction_time,symbol,first_update_id,last_update_id,previous_update_id,side,level_id,price,quantity
i64,"datetime[μs, UTC]","datetime[μs, UTC]",str,i64,i64,i64,i64,i64,"decimal[*,10]","decimal[*,10]"
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,1,1,3021.3900000000,148.4980000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,-1,1,3021.4000000000,76.8370000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,1,2,3021.3800000000,0.1640000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,-1,2,3021.4100000000,0.0140000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,1,3,3021.3700000000,0.3140000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,-1,3,3021.4200000000,0.0070000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,1,4,3021.3400000000,0.0070000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,-1,4,3021.4300000000,0.0460000000
3,2025-12-05 21:49:20.943 UTC,2025-12-05 21:49:20.941 UTC,"""ETHUSDT""",9388388935434,9388388948924,9388388934995,1,5,3021.3300000000,0.0390000000
